## Todo List
- [x] Checar se formato de datas conferem.
- [x] Remover UGs sem liberação OC
- [x] Considerar coluna montagem. Onde estiver vazio, pegar valor de Torres para EOL, no skate.
- [ ] Separar um cojunto aleatório para teste
- [ ] Calcular tempos de construção por marco: (IO_realizado - OC_realizado), (CC_realizado - OC_realizado) ...etc  (ordem:IO,CC,ME,OT)
- [ ] Calcular atraso da operação comercial (OC_realizado - OC_outorgado)
- [ ] Calcular média e dp do tempo de construção.
- [ ] Calcular data dos marcos medios (OC_outorgado - tempo_medio_de_construcao) por fonte.
- [ ] Calcular atrasos com base nos marcos medios
- [ ] Calcular media e dp dos atrasos. 
- [ ] Normalizar os atrasos calculados (atraso - atraso_medio)/dp
- [ ] Construir matriz de correlação de Pearson
- [ ] Calcular os coeficientes para regressão linear
- [ ] Para UHE pegar todas UGs, para as demais pegar a primeira UG apenas 
- [ ] Comparar pegando primeira UG e media por grupo



## Importa bibliotecas

In [1]:
import pandas as pd
import numpy as np
import re
import pyodbc
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats

##  Cria conexões com os bancos de dado

## Baixando banco de dados

### Corrigindo banco de dados bdworgs


In [2]:
def corrige_ug(ug): # Função criada para corrigir nomes das UGs
    ug = ug.replace(" ","")
    ug = ug.replace("e",",")
    ug = ug.replace("-","a")
    ug = re.sub(r",+",r",",ug)
    ug = re.sub(r"([,ae])(0)",r"\1",ug)
    ug = re.sub(r"^0",r"",ug)
    ug = ug.split("a")
    lista = []
    for i in range(len(ug)-1):
        menor = int(ug[i].split(",")[-1])
        maior = int(ug[i+1].split(",")[0])
        list_nums = list(range(menor+1,maior))
        lista += list_nums
    ug = [el.split(",") for el in ug]
    ug = [int(el2) for el1 in ug for el2 in el1]
    ug+= lista
    return (sorted(ug))

### Carrega bancos de dados

In [3]:
# Carrega bdworgs
bdworgs = pd.read_feather("./bdworgs_corrigido.feather")

# Carrega dois bancos de dados do skate
skate_ug = pd.read_feather("./skate_ug.feather")
skate_usinas = pd.read_feather("./skate_usinas.feather")

In [8]:
mask = bdworgs.Montagem_Outorga.isnull() 
bdworgs.loc[mask,"Montagem_Outorga"] = bdworgs.loc[mask,"Torres_Outorga"]

mask = bdworgs.Montagem_Outorga.isnull() 
bdworgs.loc[mask,"Montagem_Real"] = bdworgs.loc[mask,"Torres_Real"]



In [11]:
bdworgs

,Idempreendimento,Id_UG,Num_UG,Pot_UG,OpTest_Outor_UG,OpCom_Outor_UG,OpTest_R_UG,OpCom_R_UG,UF,PotenciaOutorgada,...,Montagem_Real,Enchimento_Real,Torres_Real,observacao1,observacao2,ClasseCombustivel,DscComercializacaoEnergia,DscSistema,Skate,FonteDataOC
0,30352,979,1 a 14,29.40,2013-06-10,2013-07-14,2014-02-04,2014-03-29,CE,29.40,...,None,None,2013-09-24,Relatório de enviado por e-mail em 3/2/14.,None,Eólica,ACR,Sistema Interligado Nacional,não,plan
1,30353,980,1 e 2,2.89,2014-04-04,2014-05-08,2012-05-17,2012-07-31,SC,2.89,...,2012-01-10,2012-04-25,None,None,None,Hídrica,Fora do ACR,Sistema Interligado Nacional,não,plan
2,30354,981,1,611.11,2016-01-31,2016-03-31,2016-03-23,2016-04-20,PA,11233.10,...,2013-11-12,2015-11-24,None,Dsp. 3.209/2019 OC UG 18\nDsp. 2.953/2019 OT U...,out/19 previsão da ug18 adiantada pela veloci...,Hídrica,ACR,Sistema Interligado Nacional,não,plan
3,30354,982,2,611.11,2016-03-31,2016-05-31,2016-06-28,2016-07-16,PA,11233.10,...,2013-11-12,2015-11-24,None,Dsp. 3.209/2019 OC UG 18\nDsp. 2.953/2019 OT U...,out/19 previsão da ug18 adiantada pela veloci...,Hídrica,ACR,Sistema Interligado Nacional,não,plan
4,30354,983,3,611.11,2016-05-31,2016-07-31,2016-09-20,2016-11-02,PA,11233.10,...,2013-11-12,2015-11-24,None,Dsp. 3.209/2019 OC UG 18\nDsp. 2.953/2019 OT U...,out/19 previsão da ug18 adiantada pela veloci...,Hídrica,ACR,Sistema Interligado Nacional,não,plan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2370,30350,974,1,16.00,None,None,2011-05-25,2011-06-01,RS,30.00,...,2011-04-15,None,2011-03-14,None,None,Eólica,ACR,Sistema Interligado Nacional,não,plan
2371,30350,975,2,12.00,None,None,2011-05-25,2011-07-01,RS,30.00,...,2011-04-15,None,2011-03-14,None,None,Eólica,ACR,Sistema Interligado Nacional,não,plan
2372,30350,976,3,2.00,None,None,2011-05-25,2011-08-01,RS,30.00,...,2011-04-15,None,2011-03-14,None,None,Eólica,ACR,Sistema Interligado Nacional,não,plan
2373,30351,977,1 e 4,4.00,2012-06-26,2012-06-30,2014-03-01,2014-03-29,RN,8.00,...,2012-06-28,None,2012-06-01,None,Alterada Previsão Aneel - Informações do SIGET...,Eólica,ACR,Sistema Interligado Nacional,não,plan


### Tabela com associações de nomes entre os bancos de dados

In [13]:
# Carrega tabela de associações
associassoes = pd.read_excel("associassoes.xlsx")
associassoes

,bdworgs,skate
0,Idempreendimento,IdeUsinaOutorga
1,OpTest_Outor_UG,DatInicioOpTesteOutorgada
2,OpCom_Outor_UG,DatUGInicioOpComerOutorgado
3,OpTest_R_UG,DatLiberacaoSFGTeste
4,OpCom_R_UG,DatLiberOpComerRealizado
5,NaN,DatOutorgaUsina
6,UF,UFUsina
7,NaN,SigTipoGeracao
8,PotenciaOutorgada,VlrPotenciaOutorgada
9,Obras_Outorga,DatInicioObraOutorgado


In [14]:
# Cria dicionário dos nomes das colunas para poder relacionar os nomes das colunas
# Dicionário de colunas bdworgs > skate
dict_bdworgs_skate = dict(zip(associassoes.dropna()["bdworgs"],associassoes.dropna()["skate"]))
# Dicionário de colunas skate > bdworgs
dict_skate_bdworgs = dict(zip(associassoes.dropna()["skate"],associassoes.dropna()["bdworgs"]))

In [15]:
# Colunas carregados do skate
skate_cols = [el for el in associassoes["skate"].dropna()]
# Colunas carregadas do bdworgs
bdworgs_cols =[el for el in associassoes["bdworgs"].dropna()]
skate_ug_cols = []
skate_usinas_col = []
for col in skate_cols:
    if col in skate_ug.columns:
        skate_ug_cols.append(col)
    if col in skate_usinas.columns:
        skate_usinas_col.append(col)
        
bdworgs = bdworgs[bdworgs_cols]
skate_ug = skate_ug[skate_ug_cols]
skate_usinas = skate_usinas[skate_usinas_col]
bdworgs = bdworgs.rename(columns=dict_bdworgs_skate)
bdworgs.sample(5)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,...,DatMontagemOutorgado,DatMontagemRealizado,Torres_Outorga,Torres_Real,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,Num_UG,Pot_UG
2333,30316,2012-06-01,2012-07-01,2014-06-10,2014-07-04,BA,9.6,2010-10-27,2011-07-01,2011-05-11,...,2011-09-27,2012-07-01,2011-09-27,2012-03-01,None,None,2010-12-27,2011-12-01,1 a 6,9.6
96,30480,2013-06-22,2013-07-01,2013-11-09,2014-02-28,RS,30.0,2011-10-01,2012-08-01,2012-01-01,...,2013-05-01,2013-09-06,2013-05-01,2013-09-15,None,None,2011-11-01,2012-05-01,4,3.0
451,31186,2017-02-28,2017-05-01,2016-07-01,2016-08-05,AP,219.0,2013-09-01,2013-11-01,2014-10-01,...,2015-01-02,2014-08-31,None,None,2016-12-01,2016-01-06,2015-11-01,2015-01-30,3,73.0
66,30437,2012-08-26,2012-09-25,2014-09-10,2014-10-08,BA,30.0,2012-03-01,2012-05-25,2012-05-02,...,2012-06-01,2013-02-04,2012-06-01,2012-08-06,None,None,2011-10-01,2012-04-27,1 a 15,30.0
1371,37976,2022-12-01,2023-01-01,2018-11-23,2018-12-28,MA,27.0,2022-03-01,2018-05-15,2022-06-01,...,2022-07-01,None,2022-07-01,2018-10-15,None,None,2022-08-01,2018-06-15,8 e 9,5.4


### Adiciona as seguintes  colunas no bdworgs:
- Quant_UG  &rarr; Quantidade de UGs agrupadas
- SigTipoGeracao &rarr; Tipo de geração


In [16]:
# Insere coluna Quant_UG em bdworgs
bdworgs.insert(3,"Quant_UG",bdworgs["Num_UG"].apply(lambda x : len(corrige_ug(x))))
# Remove coluna Num_UG
bdworgs = bdworgs.drop("Num_UG",axis=1)

ceg_df = pd.read_excel("./Relatorio_Outorgas.xlsx")[["Identificador da Usina","Tipo de Geração"]]
dict_id_ceg = dict(zip(ceg_df["Identificador da Usina"],ceg_df["Tipo de Geração"]))
bdworgs["SigTipoGeracao"] = bdworgs["IdeUsinaOutorga"].map(dict_id_ceg)
bdworgs.sample(5)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,Quant_UG,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,...,DatMontagemOutorgado,DatMontagemRealizado,Torres_Outorga,Torres_Real,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,Pot_UG,SigTipoGeracao
1998,29639,2009-12-01,2010-01-01,2,2010-04-24,2010-05-27,BA,148.8,2009-01-01,2009-01-15,...,2009-07-01,None,None,None,None,None,None,None,79.360,UTE
1159,33757,2017-05-08,2017-08-01,28,2018-01-31,2018-03-17,BA,28.7,2016-05-01,2017-03-15,...,2016-10-01,2017-07-15,None,None,None,None,2017-01-02,2016-11-30,28.700,UFV
788,31762,2017-12-15,2018-01-01,1,2017-04-27,2017-05-28,PE,25.3,2017-03-15,2016-05-15,...,2017-08-01,None,2017-08-01,2017-03-31,None,None,2017-04-01,2016-05-15,2.300,EOL
2261,30196,None,None,1,2014-01-30,2014-02-18,MA,176.2,2011-08-31,2012-05-15,...,2011-12-31,2012-09-17,None,None,None,None,None,None,9.341,UTE
95,30480,2013-06-22,2013-07-01,2,2013-11-09,2014-02-13,RS,30.0,2011-10-01,2012-08-01,...,2013-05-01,2013-09-06,2013-05-01,2013-09-15,None,None,2011-11-01,2012-05-01,6.000,EOL


### No dataframes skate_usina e bdworgs são feitas as seguintes alterações:
 - Onde não não existe data na coluna DatMontagemOutorgado e existe na coluna DatConclusaoTorresOutorgado, o valor que está na coluna DatConclusaoTorresOutorgado passa para a coluna DatMontagemOutorgado.
 - Mesmo processo feito para as colunas DatConclusaoTorresOutorgado/DatMontagemOutorgado é feito para as colunas DatConclusaoTorresRealizado/DatMontagemRealizado

### No BD skate_ug, linhas com todas as colunas iguais tais como IdeUsinaOutorga, DatInicioOpTesteOutorgada... (exceto  MdaPotenciaUnitaria, que não necessariamente tem que ser igual em todas as linhas)  são agrupadas e são criadas duas outras colunas: 
- Quant_UG -> &rarr; Quantidade de UGs agrupadas
- Pot_UG -> &rarr; Soma da potência das UGs que foram agrupadas

#### Entrada: skate_ug
#### Saída: skate_ug_grouped

In [19]:
skate_ug_group_list = [x for x  in skate_ug.columns.to_list() if x not in ["MdaPotenciaUnitaria"]]
skate_ug_grouped = skate_ug.groupby(skate_ug_group_list,dropna=False).agg(
    Quant_UG = ("MdaPotenciaUnitaria","count"),
    Pot_UG = ("MdaPotenciaUnitaria","sum")
    
).reset_index()

### Realiza o processo acima para o bdworgs:
- Quant_UG -> &rarr; Quantidade de UGs agrupadas. Nesse caso, como o bdworgs, já possuía a coluna Quant_UG, os valores dessa coluna foram somados para os valores agrupados.
- Pot_UG -> &rarr; Soma da potência dos conjuntos de UGs que foram agrupados.

#### Entrada: bdworgs
#### Saída: bdworgs_ug_grouped

In [20]:
dummy_bdworgs = bdworgs.rename(columns={"Quant_UG":"Quant_UG_","Pot_UG":"Pot_UG_"})
dummy_bdworgs_group_list = [x for x  in dummy_bdworgs.columns.to_list() if x not in ["Quant_UG_","Pot_UG_"]]

bdworgs_ug_grouped = dummy_bdworgs.groupby(dummy_bdworgs_group_list,dropna=False).agg(
    Quant_UG = ("Quant_UG_","sum"),
    Pot_UG = ("Pot_UG_","sum")
    
).reset_index() 

### Checa se a quantidade de potência e de usinas nos dataframes iniciais e agrupados são iguais.

In [21]:
print(skate_ug_grouped.Quant_UG.sum() - skate_ug.MdaPotenciaUnitaria.count())
print(skate_ug_grouped.Pot_UG.sum() - skate_ug.MdaPotenciaUnitaria.sum())
print(bdworgs_ug_grouped.Pot_UG.sum() - bdworgs_ug_grouped.Pot_UG.sum())
print(bdworgs_ug_grouped.Quant_UG.count() - bdworgs_ug_grouped.Quant_UG.count())


0
0.0
0.0
0


### Junta bancos de dados skate_ug e skate_usina pela coluna IdeUsinaOutorga. Todos os atributos de cada usina são passados para cada UG pertecente a usina.

#### Entrada: skate_ug_grouped & skate_usinas
#### Saída: skate_merged

In [22]:
skate_merged  = pd.merge(skate_ug_grouped.drop(["SigTipoGeracao"],axis=1),
        skate_usinas,
        on="IdeUsinaOutorga",
         how="outer")
skate_merged.sample(10)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,Quant_UG,Pot_UG,DatOutorgaUsina,UFUsina,SigTipoGeracao,...,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,DatConcretagemRealizado,DatMontagemOutorgado,DatMontagemRealizado,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado
1064,34811,2022-12-05,2022-12-20,NaN,NaN,5,31000.00,2020-11-03,BA,EOL,...,2022-05-13,2021-03-15,2022-07-15,2021-11-03,2022-08-15,None,None,None,2022-04-20,2021-05-14
1488,37102,2023-12-17,2024-01-01,2020-05-28,2020-06-16,1,4200.00,2018-12-27,BA,EOL,...,2023-01-31,2019-07-15,2023-04-01,2019-09-16,2023-08-04,2020-02-12,None,None,2023-01-31,2019-05-01
2209,40866,2022-01-01,2022-04-01,NaN,NaN,33,49999.95,2019-12-17,MG,UFV,...,2021-05-01,2021-04-15,None,None,2021-06-01,2021-09-15,None,None,2021-02-01,2021-04-15
2122,40611,2023-12-01,2024-01-01,2021-06-17,2021-07-24,1,5100.00,2019-01-28,BA,EOL,...,2022-08-01,2020-07-15,2022-12-30,2020-08-08,2023-06-01,2021-02-24,None,None,2023-06-15,2019-12-20
3017,48772,2024-12-01,2025-01-01,NaN,NaN,16,50000.00,2022-01-25,MG,UFV,...,2024-06-01,None,None,None,2024-07-01,None,None,None,2024-07-01,None
3339,50009,2023-12-30,2024-01-31,NaN,NaN,16,50000.00,2022-01-18,SE,UFV,...,2023-01-31,None,None,None,2023-03-01,None,None,None,2023-01-10,None
2387,44517,2025-11-29,2025-12-31,NaN,NaN,134,23310.64,2022-04-19,PE,UFV,...,2024-12-31,None,None,None,2025-01-29,None,None,None,2024-12-10,None
2582,46518,2022-12-30,2023-01-31,NaN,NaN,28,43814.96,2021-03-30,PI,UFV,...,2021-07-29,None,None,None,2022-01-02,None,None,None,2021-07-29,None
197,30447,NaN,2018-04-03,2018-04-07,2018-04-21,1,4500.00,2020-01-17,SP,UTE,...,None,2018-04-07,None,None,None,2018-04-07,None,None,None,2018-04-07
1701,37720,2018-11-10,2018-11-19,2019-10-19,2020-02-01,17,8836.00,2017-07-25,AM,UTE,...,2018-04-26,2018-11-15,None,None,2018-06-24,2019-05-03,None,None,2018-07-24,2019-07-01


### Converte colunas cujos nomes iniciam em "Dat" para tipo de data. É necessário que as colunas estejam nomeadas no padrão do skate.

In [23]:
colunas_datas = []
for col in bdworgs_ug_grouped.columns:
    if col[0:3] == "Dat":
        colunas_datas.append(col)
        print(col)
        skate_merged[col] = pd.to_datetime(skate_merged[col],format="%Y-%m-%d",errors="coerce")
        bdworgs_ug_grouped[col] = pd.to_datetime(bdworgs_ug_grouped[col],format="%Y-%m-%d",errors="coerce")

DatInicioOpTesteOutorgada
DatUGInicioOpComerOutorgado
DatLiberacaoSFGTeste
DatLiberOpComerRealizado
DatInicioObraOutorgado
DatInicioObraRealizado
DatConcretagemOutorgado
DatConcretagemRealizado
DatMontagemOutorgado
DatMontagemRealizado
DatEnchimentoOutorgado
DatEnchimentoRealizado
DatSisTransmissaoOutorgado
DatSisTransmissaoRealizado


### Checa quais colunas não estão presentes em cada um dos dataframes

In [24]:
print("Colunas que estão no skate, porém não no bdworgs:")
for col in skate_merged.columns:
    if col not in bdworgs.columns:
        print(col)
        
print("-"*10)
print("Colunas que estão no bdworgs, porém não no skate:")
for col in bdworgs.columns:
    if col not in skate_merged.columns:
        print(col)

Colunas que estão no skate, porém não no bdworgs:
DatOutorgaUsina
----------
Colunas que estão no bdworgs, porém não no skate:
Torres_Outorga
Torres_Real


### Junta BDs skate e bdworgs.
#### Entrada: skate_merged & bdworgs_ug_grouped
#### Saída: skate_merged

In [25]:
bdworgs_ug_grouped["origem"] = "bdworgs"
skate_merged["origem"] = "skate"
bd_combined = pd.concat([bdworgs_ug_grouped,skate_merged],axis=0,ignore_index=True)
#bd_combined = bdworgs_ug_grouped.copy(deep=True)
bd_combined["origem"] = bd_combined["origem"].astype("category")
bd_combined.sample(10)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,...,Torres_Real,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,SigTipoGeracao,Quant_UG,Pot_UG,origem,DatOutorgaUsina
5871,52397,NaT,2025-02-15,NaT,NaT,GO,50000.0,NaT,NaT,NaT,...,NaN,NaT,NaT,NaT,NaT,UFV,16,50000.0,skate,2022-02-15
4375,38395,2058-12-01,2024-01-01,NaT,NaT,CE,30000.0,2023-02-01,NaT,NaT,...,NaN,NaT,NaT,2023-03-30,NaT,UFV,24,30000.0,skate,2021-03-16
2390,27922,2018-07-07,2018-08-07,2022-05-14,NaT,SP,22000.0,2015-07-27,2021-09-15,NaT,...,NaN,NaT,NaT,2016-03-10,2019-12-01,UTE,1,10000.0,skate,2015-08-04
2434,28785,2005-10-31,2006-01-31,NaT,NaT,GO,27000.0,2004-04-30,NaT,2004-07-31,...,NaN,NaT,NaT,NaT,NaT,PCH,3,27000.0,skate,2002-12-17
2569,30982,2017-09-01,2017-10-31,NaT,NaT,SC,19499.0,2013-05-30,NaT,2013-05-30,...,NaN,2014-10-01,NaT,2013-11-30,NaT,PCH,1,6500.0,skate,2012-06-12
3060,33597,2022-03-11,2022-03-25,NaT,NaT,RN,25200.0,2020-09-21,2021-06-15,2020-11-16,...,NaN,NaT,NaT,2020-09-28,2021-06-18,EOL,1,4200.0,skate,2020-04-28
2859,32571,2024-11-30,2024-12-30,NaT,NaT,SC,5100.0,2023-12-01,NaT,2024-03-08,...,NaN,2024-09-01,NaT,2024-06-02,NaT,PCH,1,1700.0,skate,2018-02-20
3656,35764,2021-12-01,2022-02-01,2019-12-20,2020-01-31,SC,24000.0,2021-01-01,2018-06-15,2021-03-10,...,NaN,2021-12-16,2019-11-13,2021-07-01,2019-10-01,PCH,1,4800.0,skate,2018-06-01
2122,33929,2018-10-01,2018-11-01,2019-01-18,2019-02-05,BA,30.0,2018-02-01,2018-04-03,NaT,...,NaN,NaT,NaT,2018-03-01,2018-05-15,UFV,4,4.0,bdworgs,NaN
5012,46912,2024-01-01,2024-01-01,NaT,NaT,MG,48118.0,2023-03-01,NaT,NaT,...,NaN,NaT,NaT,2023-07-01,NaT,UFV,14,48118.0,skate,2021-06-22


In [26]:
bd_combined.to_parquet("bd_combined.gzip")